In [1]:
import requests
import os
from dotenv import load_dotenv
import sys
import json
import pandas as pd 
import numpy as np

sys.path.append("../")
load_dotenv()
key = os.environ.get("KEY")

url = "https://zillow-com4.p.rapidapi.com/v2/properties/search-for-rent"

In [3]:
citys = ['La Jolla', 
         'University City', 
         'Clairemont Mesa West', 
         'Clairemont Mesa East',
            'North Clairemont',
         'Sorrento Valley', 
         'Mira Mesa', 
         'Kearny Mesa', 
         'Pacific Beach',
         'Bay Ho',
         "Del Mar Heights",
         "Linda Vista"]

In [131]:
datas = {}
jsons = {}
for c in citys:
	loc = c + ", San Diego, CA"
	payload = { "location": loc}
	headers = {
		"content-type": "application/json",
		"X-RapidAPI-Key": "82aac7b40fmshe040a3f42be54a2p123dccjsn705f8900448c",
		"X-RapidAPI-Host": "zillow-com4.p.rapidapi.com"
	}

	response = requests.post(url, json=payload, headers=headers)
	jsons[c] = response.json()
	datas[c] = response.json()['data']

In [132]:
for k in citys:
    with open('data/'+k+'.json', 'w') as f:
        json.dump(jsons[k], f)


In [4]:
#data retrieve
datas ={}
for c in citys:
    f = open('data/'+c+'.json', 'r')
    datas[c] = json.load(f)['data']
    f.close()

In [5]:
fin_data = pd.DataFrame()
for c in citys:   
    temp = datas[c]
    temp = [i['property'] for i in temp]
    pre_df = {
        'address': [],
        'city':[],
        'type':[],
        'bedroom':[],
        'bathroom':[],
        'price':[]
    }
    for r in temp:
        if 'bathrooms' in r:
            pre_df['address'] += [r['address']['streetAddress']]
            pre_df['city'] += [c]
            if 'groupType' in r:
                pre_df['type'] += [r['groupType']]
            else:
                pre_df['type'] += [np.nan]
            pre_df['bathroom'] += [r['bathrooms']]
            pre_df['bedroom'] += [r['bedrooms']]
            pre_df['price'] += [r['price']['value']]
        else:
            for unit in r['unitsGroup']:
                pre_df['address'] += [r['address']['streetAddress']]
                pre_df['city'] += [c]
                if 'groupType' in r:
                    pre_df['type'] += [r['groupType']]
                else:
                    pre_df['type'] += [np.nan]
                pre_df['bathroom'] += [np.nan]
                pre_df['bedroom'] += [unit['bedrooms']]
                pre_df['price'] += [unit['minPrice']]
    df = pd.DataFrame(pre_df)
    fin_data = pd.concat([fin_data,df])


In [8]:
fin_data.to_csv('data/combined_data.csv',index=False)